<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/NLP/NLP_CharRNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char RNN

문자 단위 RNN : RNN의 입출력 단위가 단어 레벨(word-level)이 아닌 문자 레벨(Char-level)로 RNN을 구현

## 문자 단위 RNN을 다대다 구조로 구현

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN 구현

입력 데이터와 레이블 데이터에 대해서 문자 집합 생성

In [ ]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))

['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [ ]:
input_size = vocab_size # One-hot 사용할 것이므로 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [ ]:
test_list = [1,2,3,4,5]
test_value_list=['a','b','c','d','e']
test_dict = dict((c,i) for i,c in enumerate(test_value_list))
test_dict

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

In [ ]:
# 문자 집합에 고유 인덱스 부여
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
char_to_index

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}

나중에 예측 결과를 다시 문자 시퀀스로 보기위해, 정수로부터 문자를 얻을 수 있는 dict 생성

In [ ]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
index_to_char

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}

입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑

In [ ]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [ ]:
np.eye(3)[1] # np.eye(n)[m] One-hot 벡터 생성 효과

array([0., 1., 0.])

* 파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받으므로 배치를 위한 차원 추가

In [ ]:
x_data = [x_data]
y_data = [y_data]

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


데이터를 텐서로 변환

In [ ]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

텐서의 크기 확인

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape)) 
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


훈련 데이터의 크기 : torch.Size([1, 5, 5])  
레이블의 크기 : torch.Size([1, 5])  

교재에서의 결과 (배치 사이즈가 표시되어있음)

### 모델 구현

In [ ]:
class Net(torch.nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net,self).__init__()
        # RNN 셀 구현
        self.rnn = torch.nn.RNN(input_size,hidden_size,batch_first=True)
        # 출력층 구현
        self.fc = torch.nn.Linear(hidden_size,output_size,bias=True)
        
    def forward(self,x): # 구현한 RNN 셀과 출력층을 연결
        x,_status = self.rnn(x)
        x = self.fc(x)
        return x

모델 저장

In [ ]:
net = Net(input_size,hidden_size,output_size)

모델에 텐서 입력

In [ ]:
outputs = net(X)
print(outputs.shape) # 배치 차원, 시점(timesteps), 출력의 크기

torch.Size([1, 5, 5])


나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다 (즉 차원을 하나 낮춘다)

In [ ]:
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


옵티마이저, 손실 함수 정의

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6259359121322632 prediction:  [[3 3 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lllll
1 loss:  1.3896162509918213 prediction:  [[4 0 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  p!ll!
2 loss:  1.1391284465789795 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
3 loss:  0.8826513290405273 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.6482082605361938 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.4503559470176697 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.31385722756385803 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.21607990562915802 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.1435898393392563 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.0956205278635025 prediction:  [[4 4 3 2 0]

# Char RNN 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 데이터 전처리

In [ ]:
test_sentence = '가나 다라!@ a, bc'
set(test_sentence)

{' ', '!', ',', '@', 'a', 'b', 'c', '가', '나', '다', '라'}

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [ ]:
len(sentence)

180

In [ ]:
print(set(sentence))

{',', 'e', 's', 'g', 'k', 'w', 'm', 't', 'b', 'r', "'", 'i', 'n', 'u', '.', 'l', ' ', 'y', 'f', 'd', 'p', 'a', 'o', 'c', 'h'}


In [ ]:
char_set = list(set(sentence)) # 문자 집합 생성
char_dic = {c: i for i,c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic)

{',': 0, 'e': 1, 's': 2, 'g': 3, 'k': 4, 'w': 5, 'm': 6, 't': 7, 'b': 8, 'r': 9, "'": 10, 'i': 11, 'n': 12, 'u': 13, '.': 14, 'l': 15, ' ': 16, 'y': 17, 'f': 18, 'd': 19, 'p': 20, 'a': 21, 'o': 22, 'c': 23, 'h': 24}


In [ ]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [ ]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10 # 임의 지정
learning_rate = 0.1

In [ ]:
# sequence_length 단위로 샘플들을 잘라서 데이터 생성
x_data = []
y_data = []

for i in range(0,len(sentence)-sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(i,x_str,'->',y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

(len(sentence) - sequence_length) 개의 샘플 생성

In [ ]:
print(x_data[0]) # if you wan 의 정수 인코딩
print(y_data[0]) # f you want 의 정수 인코딩
# 한 칸씩 쉬프트 된 시퀀스

[11, 18, 16, 17, 22, 13, 16, 5, 21, 12]
[18, 16, 17, 22, 13, 16, 5, 21, 12, 7]


In [ ]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [ ]:
x_one_hot[0][0] # if you wan 의 i 의 원-핫 인코딩

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [ ]:
# 훈련 데이터의 첫번째 샘플
print(X[0]) # if you wan 의 원-핫 인코딩 (10개 = sequence_length) 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
# 레이블 데이터의 첫번째 샘플
print(Y[0]) # f you want 에 해당

tensor([18, 16, 17, 22, 13, 16,  5, 21, 12,  7])


### 모델 구현

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(dic_size, hidden_size, 2) # 층 2개

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [ ]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

upruppruprupduruprrpurruiupdrpuprupripprpudupuprpruppwiprirdpruprrppprduppuuruupurruprrrrprruprprdprrururuuduprdrupurrupprirupruwrupdprduruuppupprruprruruuprrruppwrurppruppuprruip
                                                                                                                                                                                   
  pat rba                                                     r                                rt   r                                                 r                     r      
                                                                                                                                                                                   
 ot ittt t i ttoi itttt itottit itti i ittt itot ttt tttititit t ittttottottiti ttttti itottti it iotttttttot itttttt tttd i itittii itoitoi it i ttedtotitititi iti t tititot it t
totktptktpodktp pppoddtkdpkppkiptpdkopkpktpdopptktktkopppipktpopdopdppktpktpopdkpptpppoupkopodpopktk

g tou want to butld a ship, don't arum up people together te collect wood and don't assign them tasks and work, but rather teach them ta long for themend ess immen,ity of thems an
g tou want to butld a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them ta long for the end ess immen,ity of theme an
g tou want to butld a ship, don't drum up people together to collect wood and don't dssign them tasks and work, but rather toach them ta long for the end ess immen,ity of the e ac
g tou want to butld a ship, don't drum up people together te collect wood and don't dssign them tasks and work, but rather teach them ta long for the end ess immenaity of thems ac
g tou want to butld a ship, don't drum up people together te collect wood and don't dssign the  tasks and work, but rather teach the  ta long for the end ess immen,ity of the e ac
g tou want to butld a ship, don't drum up people together to collect wood and don't dssign them task